# Belief Collapse: Comprehensive Analysis

This notebook consolidates the **valid, working approaches** for measuring belief collapse in transformers.

## What we measure (and why it works)

| Metric | What it captures | Why it's valid |
|--------|------------------|----------------|
| **Attention entropy** | How focused vs diffuse attention is | Direct measure of uncertainty |
| **Output probability sharpening** | How the logit distribution tightens | Direct measure of belief collapse |
| **Geometric alignment of heads** | Whether heads "agree" in vector space | No wave assumptions, pure geometry |
| **Activation change across positions** | How information propagates | Uses actual sequence axis |
| **Cosine similarity across layers** | Hidden state convergence | Measures representation stability |



In [ ]:
# Imports
import math
import json
from pathlib import Path
from typing import Dict, List, Tuple

import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import cm

from transformers import GPT2LMHeadModel, GPT2Tokenizer

NOTEBOOK_DIR = Path.cwd()
FIG_DIR = NOTEBOOK_DIR / "figs_collapse"
FIG_DIR.mkdir(exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
# Load GPT-2
model_name = "gpt2"
print(f"Loading {model_name}...")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, output_attentions=True)
model.to(device)
model.eval()

n_layers = model.config.n_layer
n_heads = model.config.n_head
d_model = model.config.n_embd
d_head = d_model // n_heads
print(f"Model: {n_layers} layers, {n_heads} heads, d_model={d_model}, d_head={d_head}")


In [ ]:
# Inference helper
def run_inference(text: str) -> Dict:
    input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
    tokens = [tokenizer.decode([tid]) for tid in input_ids[0]]
    with torch.no_grad():
        outputs = model(input_ids, output_attentions=True, output_hidden_states=True)
    return {
        "tokens": tokens,
        "input_ids": input_ids,
        "logits": outputs.logits.cpu(),
        "attentions": [a.cpu() for a in outputs.attentions],
        "hidden_states": [h.cpu() for h in outputs.hidden_states],
        "n_layers": len(outputs.attentions),
        "n_heads": outputs.attentions[0].size(1),
        "seq_len": input_ids.size(1),
    }

print("run_inference() ready")


## Metric 1: Attention entropy across layers

Directly measures how focused (low entropy) or diffuse (high entropy) the attention distribution is. No assumptions beyond the attention weights themselves.


In [ ]:
def compute_attention_entropy(attn: torch.Tensor) -> float:
    """Entropy of attention distribution. High = diffuse, Low = focused."""
    p = attn.clamp(min=1e-10)
    entropy = -(p * p.log()).sum(dim=-1)
    return entropy.mean().item()

def attention_entropy_per_layer(result: Dict, query_pos: int = -1) -> List[float]:
    """Attention entropy at each layer for a given query position."""
    entropies = []
    for attn in result["attentions"]:
        # attn: [1, heads, seq, seq]
        q_attn = attn[0, :, query_pos, :]  # [heads, seq]
        entropies.append(compute_attention_entropy(q_attn))
    return entropies

print("attention_entropy functions ready")


## Metric 2: Output probability sharpening (logit lens)

Apply the output projection at each layer to see how the probability distribution evolves. Lower entropy = sharper = more confident.


In [ ]:
def logit_lens(result: Dict, pos: int = -1) -> Tuple[List[float], List[str]]:
    """Apply output projection to hidden states at each layer.
    
    Returns:
        entropies: Output entropy at each layer
        top_tokens: Most likely token at each layer
    """
    # Get the language model head (unembedding)
    lm_head = model.lm_head
    ln_f = model.transformer.ln_f  # final layer norm
    
    entropies = []
    top_tokens = []
    
    for layer_idx, hidden in enumerate(result["hidden_states"]):
        # hidden: [1, seq, d_model]
        h = hidden[0, pos, :]  # [d_model]
        
        # Apply final layer norm (for proper comparison)
        h_normed = ln_f(h.to(device))
        
        # Project to vocab
        logits = lm_head(h_normed)  # [vocab]
        probs = F.softmax(logits, dim=-1)
        
        # Entropy
        p = probs.clamp(min=1e-10)
        entropy = -(p * p.log()).sum().item()
        entropies.append(entropy)
        
        # Top token
        top_id = probs.argmax().item()
        top_tokens.append(tokenizer.decode([top_id]))
    
    return entropies, top_tokens

print("logit_lens() ready")


## Metric 3: Geometric alignment of head outputs

Measure whether attention heads "agree" by computing pairwise cosine similarity between head attention-weighted contexts. High agreement = heads are aligned.


In [ ]:
def head_context_vectors(result: Dict, layer_idx: int, query_pos: int = -1) -> np.ndarray:
    """Compute attention-weighted context vector for each head.
    
    Returns:
        contexts: [n_heads, d_model] array of context vectors
    """
    attn = result["attentions"][layer_idx][0].numpy()  # [heads, seq, seq]
    hidden_in = result["hidden_states"][layer_idx][0].numpy()  # [seq, d_model]
    
    contexts = []
    for h in range(attn.shape[0]):
        weights = attn[h, query_pos, :]  # [seq]
        ctx = weights @ hidden_in  # [d_model]
        contexts.append(ctx)
    return np.stack(contexts, axis=0)

def head_alignment(result: Dict, layer_idx: int, query_pos: int = -1) -> float:
    """Mean pairwise cosine similarity between head context vectors."""
    contexts = head_context_vectors(result, layer_idx, query_pos)
    n = contexts.shape[0]
    
    # Normalize
    norms = np.linalg.norm(contexts, axis=1, keepdims=True) + 1e-10
    contexts_normed = contexts / norms
    
    # Pairwise cosine similarity
    sim_matrix = contexts_normed @ contexts_normed.T
    
    # Mean of upper triangle (excluding diagonal)
    upper = sim_matrix[np.triu_indices(n, k=1)]
    return float(upper.mean())

def head_alignment_per_layer(result: Dict, query_pos: int = -1) -> List[float]:
    """Head alignment at each layer."""
    return [head_alignment(result, l, query_pos) for l in range(result["n_layers"])]

print("head_alignment functions ready")


## Metric 4: Activation change across token positions

Measures how much the hidden state changes from one token position to the next. Uses the actual sequence axis (not embedding dimension).


In [ ]:
def activation_change_across_positions(result: Dict, layer_idx: int) -> np.ndarray:
    """Cosine distance between consecutive position hidden states.
    
    Returns:
        changes: [seq_len-1] array of cosine distances
    """
    hidden = result["hidden_states"][layer_idx][0].numpy()  # [seq, d_model]
    seq_len = hidden.shape[0]
    
    if seq_len < 2:
        return np.array([])
    
    # Normalize
    norms = np.linalg.norm(hidden, axis=1, keepdims=True) + 1e-10
    hidden_normed = hidden / norms
    
    # Consecutive cosine similarity -> distance
    changes = []
    for i in range(seq_len - 1):
        sim = np.dot(hidden_normed[i], hidden_normed[i + 1])
        changes.append(1 - sim)  # distance = 1 - similarity
    
    return np.array(changes)

def mean_position_change_per_layer(result: Dict) -> List[float]:
    """Mean activation change across positions at each layer."""
    changes = []
    for l in range(len(result["hidden_states"])):
        c = activation_change_across_positions(result, l)
        changes.append(float(c.mean()) if len(c) > 0 else 0.0)
    return changes

print("activation_change functions ready")


## Metric 5: Cosine similarity of hidden states across layers

Measures how much the representation at a given position changes from layer to layer. Low change late = convergence/stability.


In [ ]:
def layer_to_layer_similarity(result: Dict, pos: int = -1) -> List[float]:
    """Cosine similarity between consecutive layers at a given position.
    
    Returns:
        sims: [n_layers] similarities (first is embedding -> layer0)
    """
    hidden_states = result["hidden_states"]  # len = n_layers + 1 (emb + layers)
    sims = []
    
    for i in range(len(hidden_states) - 1):
        h1 = hidden_states[i][0, pos, :].numpy()
        h2 = hidden_states[i + 1][0, pos, :].numpy()
        
        # Cosine similarity
        norm1 = np.linalg.norm(h1) + 1e-10
        norm2 = np.linalg.norm(h2) + 1e-10
        sim = np.dot(h1, h2) / (norm1 * norm2)
        sims.append(float(sim))
    
    return sims

def layer_similarity_to_final(result: Dict, pos: int = -1) -> List[float]:
    """Cosine similarity between each layer and the final layer representation.
    
    Returns:
        sims: [n_layers+1] similarities to final layer
    """
    hidden_states = result["hidden_states"]
    final = hidden_states[-1][0, pos, :].numpy()
    final_norm = np.linalg.norm(final) + 1e-10
    
    sims = []
    for i in range(len(hidden_states)):
        h = hidden_states[i][0, pos, :].numpy()
        h_norm = np.linalg.norm(h) + 1e-10
        sim = np.dot(h, final) / (h_norm * final_norm)
        sims.append(float(sim))
    
    return sims

print("layer_similarity functions ready")


## Prompts for testing


In [ ]:
prompts = {
    # Factual (constrained, high confidence expected)
    "factual_capital": "The capital of France is",
    "factual_sun": "The sun rises in the",
    "factual_math": "Two plus two equals",
    
    # Technical (specialized vocabulary)
    "technical_quantum": "The quantum mechanical wave function describes probability amplitudes",
    "technical_code": "Write a Python function that returns the greatest common divisor",
    
    # Open-ended (many valid continuations)
    "open_future": "In the distant future,",
    "open_meaning": "The meaning of life is",
    "open_door": "He opened the door and saw",
    
    # Ambiguous/contradictory
    "ambiguous_circle": "Describe the color of a square circle",
    "riddle": "I speak without a mouth and hear without ears; what am I?",
}

print(f"Loaded {len(prompts)} prompts")


In [ ]:
# Run comprehensive analysis on all prompts
all_results = {}

for name, prompt in prompts.items():
    print(f"Processing: {name}...")
    result = run_inference(prompt)
    
    # Compute all metrics
    attn_entropy = attention_entropy_per_layer(result)
    logit_entropy, top_tokens = logit_lens(result)
    alignment = head_alignment_per_layer(result)
    pos_change = mean_position_change_per_layer(result)
    layer_sim = layer_to_layer_similarity(result)
    sim_to_final = layer_similarity_to_final(result)
    
    # Final output
    final_logits = result["logits"][0, -1, :]
    probs = F.softmax(final_logits, dim=-1)
    final_entropy = -(probs * probs.clamp(min=1e-10).log()).sum().item()
    top_token = tokenizer.decode([probs.argmax().item()])
    
    all_results[name] = {
        "prompt": prompt,
        "tokens": result["tokens"],
        "top_token": top_token,
        "final_entropy": final_entropy,
        "attn_entropy": attn_entropy,
        "logit_entropy": logit_entropy,
        "top_tokens_by_layer": top_tokens,
        "head_alignment": alignment,
        "position_change": pos_change,
        "layer_to_layer_sim": layer_sim,
        "sim_to_final": sim_to_final,
    }

print("\nDone. All metrics computed.")


In [ ]:
# Summary table
print("="*100)
print(f"{'Prompt':<30} {'Top Token':>12} {'Entropy':>8} {'Align(L11)':>10} {'SimFinal(L0)':>12}")
print("="*100)

for name, r in all_results.items():
    prompt_short = r["prompt"][:28]
    top = r["top_token"].strip()[:10]
    ent = r["final_entropy"]
    align = r["head_alignment"][-1]
    sim = r["sim_to_final"][0]
    print(f"{prompt_short:<30} {top:>12} {ent:>8.2f} {align:>10.3f} {sim:>12.3f}")

print("="*100)


## Visualization 1: All metrics for a single interesting prompt


In [ ]:
# All metrics for the quantum prompt
target = "technical_quantum"
r = all_results[target]

fig = plt.figure(figsize=(16, 12))
gs = GridSpec(3, 2, figure=fig)

layers = list(range(n_layers))

# Attention entropy
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(layers, r["attn_entropy"], 'o-', color='crimson', linewidth=2)
ax1.set_xlabel("Layer")
ax1.set_ylabel("Entropy (nats)")
ax1.set_title("Attention Entropy (lower = more focused)")
ax1.grid(True, alpha=0.3)

# Logit lens entropy
ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(range(len(r["logit_entropy"])), r["logit_entropy"], 's-', color='darkorange', linewidth=2)
ax2.set_xlabel("Layer (0=embedding)")
ax2.set_ylabel("Output Entropy")
ax2.set_title("Logit Lens: Output Sharpening")
ax2.grid(True, alpha=0.3)

# Head alignment
ax3 = fig.add_subplot(gs[1, 0])
ax3.plot(layers, r["head_alignment"], '^-', color='forestgreen', linewidth=2)
ax3.set_xlabel("Layer")
ax3.set_ylabel("Mean Pairwise Cosine Sim")
ax3.set_title("Head Alignment (higher = more agreement)")
ax3.grid(True, alpha=0.3)

# Similarity to final layer
ax4 = fig.add_subplot(gs[1, 1])
ax4.plot(range(len(r["sim_to_final"])), r["sim_to_final"], 'D-', color='purple', linewidth=2)
ax4.set_xlabel("Layer (0=embedding)")
ax4.set_ylabel("Cosine Sim to Final")
ax4.set_title("Convergence to Final Representation")
ax4.grid(True, alpha=0.3)

# Layer-to-layer similarity
ax5 = fig.add_subplot(gs[2, 0])
ax5.plot(range(len(r["layer_to_layer_sim"])), r["layer_to_layer_sim"], 'p-', color='teal', linewidth=2)
ax5.set_xlabel("Transition (L_i -> L_{i+1})")
ax5.set_ylabel("Cosine Sim")
ax5.set_title("Layer-to-Layer Stability")
ax5.grid(True, alpha=0.3)

# Top tokens by layer
ax6 = fig.add_subplot(gs[2, 1])
ax6.axis('off')
top_tok_str = "\n".join([f"L{i}: '{t.strip()[:15]}'" for i, t in enumerate(r["top_tokens_by_layer"])])
ax6.text(0.1, 0.9, f"Top token at each layer:\n\n{top_tok_str}", fontsize=9, va='top', family='monospace')
ax6.set_title(f"Final prediction: '{r['top_token'].strip()}'")

plt.suptitle(f"Comprehensive Collapse Metrics: '{r['prompt'][:60]}...'", fontsize=14)
plt.tight_layout()
plt.savefig(FIG_DIR / "01_single_prompt_all_metrics.png", dpi=150)
plt.show()


## Visualization 2: Compare factual vs open-ended prompts


In [ ]:
# Compare factual vs open-ended
factual_keys = [k for k in all_results.keys() if k.startswith("factual")]
open_keys = [k for k in all_results.keys() if k.startswith("open")]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Attention entropy
ax1 = axes[0, 0]
for k in factual_keys:
    ax1.plot(all_results[k]["attn_entropy"], 'o-', alpha=0.7, label=k)
for k in open_keys:
    ax1.plot(all_results[k]["attn_entropy"], 's--', alpha=0.7, label=k)
ax1.set_xlabel("Layer"); ax1.set_ylabel("Entropy")
ax1.set_title("Attention Entropy: Factual (solid) vs Open (dashed)")
ax1.legend(fontsize=8); ax1.grid(True, alpha=0.3)

# Logit lens
ax2 = axes[0, 1]
for k in factual_keys:
    ax2.plot(all_results[k]["logit_entropy"], 'o-', alpha=0.7, label=k)
for k in open_keys:
    ax2.plot(all_results[k]["logit_entropy"], 's--', alpha=0.7, label=k)
ax2.set_xlabel("Layer"); ax2.set_ylabel("Output Entropy")
ax2.set_title("Logit Lens: Output Sharpening")
ax2.legend(fontsize=8); ax2.grid(True, alpha=0.3)

# Head alignment
ax3 = axes[1, 0]
for k in factual_keys:
    ax3.plot(all_results[k]["head_alignment"], 'o-', alpha=0.7, label=k)
for k in open_keys:
    ax3.plot(all_results[k]["head_alignment"], 's--', alpha=0.7, label=k)
ax3.set_xlabel("Layer"); ax3.set_ylabel("Head Alignment")
ax3.set_title("Head Alignment")
ax3.legend(fontsize=8); ax3.grid(True, alpha=0.3)

# Convergence to final
ax4 = axes[1, 1]
for k in factual_keys:
    ax4.plot(all_results[k]["sim_to_final"], 'o-', alpha=0.7, label=k)
for k in open_keys:
    ax4.plot(all_results[k]["sim_to_final"], 's--', alpha=0.7, label=k)
ax4.set_xlabel("Layer"); ax4.set_ylabel("Sim to Final")
ax4.set_title("Convergence to Final Representation")
ax4.legend(fontsize=8); ax4.grid(True, alpha=0.3)

plt.suptitle("Factual (constrained) vs Open-ended (unconstrained) Prompts", fontsize=14)
plt.tight_layout()
plt.savefig(FIG_DIR / "02_factual_vs_open.png", dpi=150)
plt.show()


## Visualization 3: Correlations and scatter plots


In [ ]:
# Scatter plots: confidence vs various metrics
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

names = list(all_results.keys())
final_entropies = [all_results[n]["final_entropy"] for n in names]
final_alignments = [all_results[n]["head_alignment"][-1] for n in names]
final_sim_to_final = [all_results[n]["sim_to_final"][0] for n in names]  # embedding sim to final
attn_entropy_drop = [all_results[n]["attn_entropy"][0] - all_results[n]["attn_entropy"][-1] for n in names]

# Color by type
colors = []
for n in names:
    if "factual" in n: colors.append("blue")
    elif "open" in n: colors.append("green")
    elif "technical" in n: colors.append("purple")
    else: colors.append("red")

# Entropy vs Head Alignment
ax1 = axes[0, 0]
ax1.scatter(final_entropies, final_alignments, c=colors, s=80, edgecolors='black')
for i, n in enumerate(names):
    ax1.annotate(n[:8], (final_entropies[i], final_alignments[i]), fontsize=7)
ax1.set_xlabel("Final Output Entropy")
ax1.set_ylabel("Final Layer Head Alignment")
ax1.set_title("Confidence vs Head Agreement")
ax1.grid(True, alpha=0.3)

# Entropy vs Sim to Final
ax2 = axes[0, 1]
ax2.scatter(final_entropies, final_sim_to_final, c=colors, s=80, edgecolors='black')
ax2.set_xlabel("Final Output Entropy")
ax2.set_ylabel("Embedding Sim to Final Layer")
ax2.set_title("Confidence vs Representation Shift")
ax2.grid(True, alpha=0.3)

# Entropy vs Attention entropy drop
ax3 = axes[1, 0]
ax3.scatter(final_entropies, attn_entropy_drop, c=colors, s=80, edgecolors='black')
ax3.set_xlabel("Final Output Entropy")
ax3.set_ylabel("Attention Entropy Drop (L0 - L11)")
ax3.set_title("Confidence vs Attention Focusing")
ax3.grid(True, alpha=0.3)

# Legend
ax4 = axes[1, 1]
ax4.axis('off')
ax4.scatter([], [], c='blue', s=80, label='Factual')
ax4.scatter([], [], c='green', s=80, label='Open-ended')
ax4.scatter([], [], c='purple', s=80, label='Technical')
ax4.scatter([], [], c='red', s=80, label='Ambiguous/Riddle')
ax4.legend(loc='center', fontsize=12)
ax4.set_title("Prompt Types")

plt.suptitle("Correlations: Output Entropy vs Collapse Metrics", fontsize=14)
plt.tight_layout()
plt.savefig(FIG_DIR / "03_correlations.png", dpi=150)
plt.show()


## Visualization 4: Logit lens token evolution


In [ ]:
# Logit lens: show how the top token changes through layers
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

selected = ["factual_capital", "open_meaning", "technical_quantum", "riddle"]

for idx, name in enumerate(selected):
    ax = axes[idx // 2, idx % 2]
    r = all_results[name]
    
    layers_ll = range(len(r["logit_entropy"]))
    ax.plot(layers_ll, r["logit_entropy"], 'o-', color='darkorange', linewidth=2)
    ax.set_xlabel("Layer (0=embedding)")
    ax.set_ylabel("Output Entropy")
    ax.set_title(f"'{r['prompt'][:40]}...'")
    ax.grid(True, alpha=0.3)
    
    # Annotate with top tokens
    for i, tok in enumerate(r["top_tokens_by_layer"]):
        if i % 3 == 0 or i == len(r["top_tokens_by_layer"]) - 1:
            ax.annotate(tok.strip()[:8], (i, r["logit_entropy"][i]), fontsize=7, rotation=30)

plt.suptitle("Logit Lens: Top Token and Entropy Through Layers", fontsize=14)
plt.tight_layout()
plt.savefig(FIG_DIR / "04_logit_lens_evolution.png", dpi=150)
plt.show()


## Visualization 5: Heatmap of all metrics across all prompts


In [ ]:
# Heatmap: head alignment across all prompts and layers
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Head alignment heatmap
alignment_matrix = np.array([all_results[n]["head_alignment"] for n in names])
ax1 = axes[0]
im1 = ax1.imshow(alignment_matrix, aspect='auto', cmap='viridis')
ax1.set_yticks(range(len(names)))
ax1.set_yticklabels([n[:12] for n in names], fontsize=8)
ax1.set_xlabel("Layer")
ax1.set_title("Head Alignment by Prompt and Layer")
plt.colorbar(im1, ax=ax1)

# Logit entropy heatmap
logit_matrix = np.array([all_results[n]["logit_entropy"] for n in names])
ax2 = axes[1]
im2 = ax2.imshow(logit_matrix, aspect='auto', cmap='magma')
ax2.set_yticks(range(len(names)))
ax2.set_yticklabels([n[:12] for n in names], fontsize=8)
ax2.set_xlabel("Layer (0=embedding)")
ax2.set_title("Output Entropy by Prompt and Layer")
plt.colorbar(im2, ax=ax2)

plt.suptitle("Metric Heatmaps Across All Prompts", fontsize=14)
plt.tight_layout()
plt.savefig(FIG_DIR / "05_heatmaps.png", dpi=150)
plt.show()


In [ ]:
# Save results
summary = {
    "model": model_name,
    "n_layers": n_layers,
    "n_heads": n_heads,
    "prompts": {
        name: {
            "prompt": r["prompt"],
            "tokens": r["tokens"],
            "top_token": r["top_token"],
            "final_entropy": float(r["final_entropy"]),
            "attn_entropy": [float(x) for x in r["attn_entropy"]],
            "logit_entropy": [float(x) for x in r["logit_entropy"]],
            "head_alignment": [float(x) for x in r["head_alignment"]],
            "position_change": [float(x) for x in r["position_change"]],
            "layer_to_layer_sim": [float(x) for x in r["layer_to_layer_sim"]],
            "sim_to_final": [float(x) for x in r["sim_to_final"]],
        }
        for name, r in all_results.items()
    }
}

with open(FIG_DIR / "collapse_results.json", "w") as f:
    json.dump(summary, f, indent=2)

print(f"Results saved to {FIG_DIR / 'collapse_results.json'}")


## Summary

### What these experiments measure (and why they're valid)

| Metric | What it shows | Validity |
|--------|---------------|----------|
| **Attention entropy** | How focused attention is at each layer | Direct measure, no assumptions |
| **Logit lens** | How the output distribution evolves through layers | Uses model's own projection |
| **Head alignment** | Whether attention heads agree (cosine similarity) | Pure geometry, no wave assumptions |
| **Sim to final** | How early layers relate to final representation | Measures convergence |
| **Layer-to-layer similarity** | Stability of representation changes | Measures transformation magnitude |

### Key findings

1. **Attention entropy** tends to decrease through layers (attention becomes more focused)
2. **Output entropy** drops through layers (belief sharpens) - visible in logit lens
3. **Head alignment** shows whether heads are computing similar things
4. **Similarity to final** shows how quickly representations converge to the output

### Figures saved

- `01_single_prompt_all_metrics.png`: All metrics for one prompt
- `02_factual_vs_open.png`: Compare constrained vs open-ended prompts
- `03_correlations.png`: Scatter plots of confidence vs metrics
- `04_logit_lens_evolution.png`: Top token evolution through layers
- `05_heatmaps.png`: Heatmaps across all prompts

### Connection to AKIRA theory

These metrics validate the "belief collapse" narrative without requiring wave/phase assumptions:
- Entropy drop = belief concentration
- Head alignment = collective agreement
- Convergence to final = representation stabilization

The collapse is real. The "phase" metaphor is intuition, not mechanism.
